In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import string 
import re
from scipy import spatial
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import pos_tag
import os
from sklearn.naive_bayes import GaussianNB

In [2]:
# example1=["I loved this movie since I was 7 and I saw it on the opening day. It was so touching and beautiful. I strongly recommend seeing for all. It's a movie to watch with your family by far.",
#           "My MPAA rating: PG-13 for thematic elements, prolonged scenes of disastor, nudity/sexuality and some language.",
#           "First things first Edison Chen did a fantastic believable job as a Cambodian hit-man born and bred in the dumps and a gladiatorial ring where he honed his craft of savage battery in order to survive living on the mantra of kill or be killed.",
#           "In a role that had little dialogue or at least a few lines in Cambodian/Thai his performance is compelling  probably what should have been in the Jet Li vehicle Danny the Dog, where a man is bred for the sole purpose of fighting and on someone else's leash.",
#           "Like Danny the Dog the much talked about bare knuckle fight sequences are not choreographed stylistically but rather designed as normal brutal fisticuffs where everything goes. This probably brought a sense of realism and grit when you see the characters slug it out at each other's throats in defending their own lives while taking it away from others. It's a grim gritty and dark movie both literally and figuratively and this sets it apart from the usual run off the mill cop thriller production.",
#           "I study at Thapar Institute of Engineering and Technology Patiala. I am a prefinal year student there."]


In [3]:
import numpy as np
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stopword = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
print(stopword)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
def get_synonyms(word):    
    synonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)
def synonym_replacement(word):

  synonyms = get_synonyms(word)
  if len(synonyms) >= 1:
    synonym = random.choice(list(synonyms))
    return synonym
  return word

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
def get_tf_idf_augment(text):
  x=len(text)
  tfidf=TfidfVectorizer()
  vc=tfidf.fit_transform(text).toarray()
  print(vc)
  #print(tfidf.vocabulary_)
  words_lib=tfidf.vocabulary_
  new_corpus=[]
  for i in range(x):
    words=text[i]
    words=words.replace(".","")
    #print(words)  
    words=words.split(" ")
    new_words=""
    for m in words:
      if m.lower() not in stopword:
        if words_lib.__contains__(m):
          if vc[i][words_lib[m.lower()]]<0.20:
              replaced_word=synonym_replacement(m)
              new_words+=(replaced_word)
          else:
            new_words+=(m.lower())
        else:
          new_words+=m.lower()
      else:
        new_words+=(m.lower())
      new_words+=" "
    #new_words=(str(new_words))
    new_corpus.append(new_words)
  return new_corpus 

In [7]:
df=pd.read_csv('/content/drive/MyDrive/PRISM/IMDB Dataset.csv')

In [8]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
X=df['review']
y=df['sentiment']

In [10]:
X=X.values.tolist()

In [11]:
X=X[:25000]
y=y[:25000]

In [12]:
X[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [13]:
Augmented_text=get_tf_idf_augment(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
Augmented_text[0]

"one of the other commentator has refer that after keep an eye on just 1 oz instalment you'll be snare they are right, as this is on the dot what find with me<br /><br />the number 1 matter that chance on me about oz was its savagery and unshrinking shot of violence, which congeal in decently from the give voice go trust me, this is not a express for the fainthearted hearted or faint this record extract no puncher with affect to drugs, excite or fury its is hardcore, in the greco roman utilise of the word<br /><br />it is ring oz as that is the moniker contribute to the oswald maximum security state penitentary it concenter in the main on emerald city, an observational surgical incision of the prison house where all the electric cell have drinking glass movement and aspect inwards, so secrecy is not eminent on the order of business em city is menage to manyaryans, muslims, gangstas, latinos, christians, italians, irish and moreso scuffles, demise stares, wily grapple and shadowed arran

In [15]:
en_stopwords=set(stopwords.words('english'))
wn = nltk.WordNetLemmatizer()

In [16]:
def getCleanedReview(review):
    review=review.lower()
    text_lc = "".join([word.lower() for word in review if word not in string.punctuation])
    #print(text_lc)
    text_rc=re.sub('[0-9]+',' ',text_lc)
    #tokenize
    tokens=word_tokenize(text_rc,language='english')
    #print(tokens)
    new_tokens=[token for token in tokens if token not in en_stopwords]
    #print(new_tokens)
    lemmatized_tokens=[wn.lemmatize(token) for token in new_tokens]
    return lemmatized_tokens

In [17]:
original=[]
X_train=[]
X_augmented_train=[]

In [18]:
for i in range(len(X)):
    X_train.append(getCleanedReview(X[i]))
    X_augmented_train.append(getCleanedReview(Augmented_text[i]))
    if y[i] == 'positive':
        original.append(1)
    else:
        original.append(0)


In [19]:
df_new=[]
df_new=pd.DataFrame(df_new)


In [20]:
df_new["cleaned_review"]=X_train
X_train=df_new["cleaned_review"]
df_new["cleaned_augmented_review"]=X_augmented_train
X_augmented_train=df_new["cleaned_augmented_review"]

In [21]:
df_new.head()

,cleaned_review,cleaned_augmented_review
0,"[one, reviewer, mentioned, watching, oz, episo...","[one, commentator, refer, keep, eye, oz, insta..."
1,"[wonderful, little, production, br, br, filmin...","[fantastic, short, output, br, br, take, profi..."
2,"[thought, wonderful, way, spend, time, hot, su...","[retrieve, howling, agency, expend, prison, te..."
3,"[basically, there, family, little, boy, jake, ...","[basically, there, household, minuscule, son, ..."
4,"[petter, matteis, love, time, money, visually,...","[petter, matteis, love, time, money, visually,..."


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import sklearn.metrics

In [38]:
X_train.dropna()
cvt = CountVectorizer(analyzer=getCleanedReview,max_features=20000)
X_v = cvt.fit_transform(X_train.apply(lambda x: np.str_(x)))
X_augmented_v=cvt.fit_transform(X_augmented_train.apply(lambda x: np.str_(x)))

In [39]:
y_train=np.array(original)

In [40]:
gnb=GaussianNB()
X_v=X_v.toarray()
X_augmented_v=X_augmented_v.toarray()
gnb.fit(X_v,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [41]:
gnb2=GaussianNB()
gnb2.fit(X_augmented_v,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [42]:
X_test=df['review'].iloc[25000:50000]
y_test=df['sentiment'].iloc[25000:50000]

In [43]:
print(y_test)

25000    negative
25001    negative
25002    positive
25003    positive
25004    negative
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 25000, dtype: object


In [44]:
X_test=X_test.values.tolist()

In [45]:
X_test_new=[]
y_test_new=[]

In [46]:
for i in range(len(X_test)):
    X_test_new.append(getCleanedReview(X_test[i]))
    if y_test[25000+i] == 'positive':
        y_test_new.append(1)
    else:
        y_test_new.append(0)

In [47]:
df_new["cleaned_test_review"]=X_test_new
X_test=df_new["cleaned_test_review"]

In [48]:
X_test.dropna()
cvt1 = CountVectorizer(analyzer=getCleanedReview,max_features=20000)
X_test_v = cvt1.fit_transform(X_test.apply(lambda x: np.str_(x)))

In [49]:
y_test=np.array(y_test_new)

In [50]:
X_test_v=X_test_v.toarray()

In [51]:
gnb.score(X_test_v,y_test)

0.49884

In [52]:
gnb2.score(X_test_v,y_test)

0.49924